In [ ]:
#data.ConceptDriftStream di scikit-multiflow

In [391]:
import skmultiflow as sk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.lof import LOF
import csv
import time

dataset=pd.read_csv("SRdrift.csv" ,usecols=["A","B","C"])
verita=pd.read_csv("SRdrift.csv" ,usecols=["V"])
dataset_np= dataset.to_numpy()
verita_np=verita.to_numpy()
store_risultati="SRdrift_risultati_25v4.csv"

In [5]:
verita_list=[]
for elem in verita_np:
    verita_list.append(elem[0])

In [6]:
dataset_list=dataset_np.tolist()


In [7]:
#feature aggiuntiva per il rilevamento del concept drif
for i in range(0, len(dataset_list)):
    b=dataset_list[i]
    new_f=0
    for j in range(0, len(dataset_list[i])):
        new_f=new_f+dataset_list[i][j]**2
    b.append(new_f)
    dataset_list[i]=b

print(dataset_list[0])       

[-258.93763863437493, -654.1949132579581, -570.7438662976431, 820768.2461505153]


In [8]:
dataset_np2=np.array(dataset_list)
print(dataset_np)
print(dataset_np2)

[[-258.93763863 -654.19491326 -570.7438663 ]
 [-258.70523191 -655.15165221 -570.92558994]
 [-259.4375307  -653.76023233 -570.98461852]
 ...
 [-258.82138476 -654.07704964 -571.45474237]
 [-259.22645187 -654.94091298 -570.51667721]
 [-259.7429683  -654.39650835 -572.13457074]]
[[-2.58937639e+02 -6.54194913e+02 -5.70743866e+02  8.20768246e+05]
 [-2.58705232e+02 -6.55151652e+02 -5.70925590e+02  8.22108114e+05]
 [-2.59437531e+02 -6.53760232e+02 -5.70984619e+02  8.20733708e+05]
 ...
 [-2.58821385e+02 -6.54077050e+02 -5.71454742e+02  8.21365819e+05]
 [-2.59226452e+02 -6.54940913e+02 -5.70516677e+02  8.21635232e+05]
 [-2.59742968e+02 -6.54396508e+02 -5.72134571e+02  8.23039167e+05]]


In [9]:
#variabili modello
max_model=5
fine=False
n_modelli_usati=0
n_conceptd_drift_dati=0
n_riutilizzo_modello=0
i_m=0 #modello attualmente in uso
f=0 #flag anomalia dati
g=0# flag anomalia modelli
n_batch=0
n_features= len(dataset_np2[0])
print(n_features)
len_W=25 #dimensione finestra dati 
len_CD=len_W*2 #dimensione finestra rilevamento concept drift
#n_neig=int(len_W/2)
n_neig=25
kswin_cf=[] #flag per feature di rilevamento concept drift
kswin_lof_model=[]
kswin_lof_model_cf=[] 
finestra_del_concetto=[]
dati_di_addestramento=[]
predizioni=[]
#utilizzo_modelli=[]
for i in range(0,n_features):
    kswin_cf.append(False)
print(kswin_cf)

4
[False, False, False, False]


In [10]:
#componenti modello
data_stream_cd= sk.data.DataStream(dataset_np2,y=None,target_idx=11)
data_stream_cd.print_df()
data_stream_cd.prepare_for_use()

data_stream= sk.data.DataStream(dataset_np,y=None,target_idx=11)
data_stream.print_df()
data_stream.prepare_for_use()

kswin=[]
#kswin_model=sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W)
for i in range(0,n_features):
    kswin.append(sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W))

#Model= LOF(n_neighbors=35, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, contamination=0.1, n_jobs=1)
Lof_models=[]

print(len(kswin))

[[-2.58937639e+02 -6.54194913e+02 -5.70743866e+02  8.20768246e+05]
 [-2.58705232e+02 -6.55151652e+02 -5.70925590e+02  8.22108114e+05]
 [-2.59437531e+02 -6.53760232e+02 -5.70984619e+02  8.20733708e+05]
 ...
 [-2.58821385e+02 -6.54077050e+02 -5.71454742e+02  8.21365819e+05]
 [-2.59226452e+02 -6.54940913e+02 -5.70516677e+02  8.21635232e+05]
 [-2.59742968e+02 -6.54396508e+02 -5.72134571e+02  8.23039167e+05]]
[]
[[-258.93763863 -654.19491326 -570.7438663 ]
 [-258.70523191 -655.15165221 -570.92558994]
 [-259.4375307  -653.76023233 -570.98461852]
 ...
 [-258.82138476 -654.07704964 -571.45474237]
 [-259.22645187 -654.94091298 -570.51667721]
 [-259.7429683  -654.39650835 -572.13457074]]
[]
4


c:\Users\Salvatore Drago\Desktop\progetto_lab\lib\site-packages\skmultiflow\data\base_stream.py:194: FutureWarning: 'prepare_for_use' has been deprecated in v0.5.0 and will be removed in v0.7.0.
New instances of the Stream class are now ready to use after instantiation.
  category=FutureWarning)


In [ ]:
'''
while fine!=True:  #no CDD
    n_batch+=1
    dati_cd=data_stream_cd.next_sample(batch_size=len_W)[0]
    dati=data_stream.next_sample(batch_size=len_W)[0]
    for i in range(0,len(kswin_cf)):
        kswin_cf[i]=False
    f=0
    

    if len(dati_cd)!=0:
        
        for i in range(0, len(kswin)):
            for dato in dati_cd:
                kswin[i].add_element(dato[i])
                if kswin[i].detected_change()==True:
                    kswin_cf[i]=True
        for i in range(0, len(kswin_cf)):
            if kswin_cf[i]==True:
                print(f'Drift zone has been detected on feature {i} in batch {n_batch} with kswin')
            else:
                print(f'No Drift zone has been detected on feature {i} in batch {n_batch} with kswin')
        

        if n_batch==1:
            print(f"Primo batch, creazione modello di base:{i_m}")
            i_m=0
            kswin_lof_model_cf.append(False)
            kswin_lof_model.append(sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W))
            Lof_models.append(LOF(n_neighbors=n_neig, algorithm='ball_tree', leaf_size=30, metric='minkowski', p=3, metric_params=None, contamination=0.1, n_jobs=1))
            Lof_models[i_m].fit(dati)
            y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
            finestra_del_concetto.append(y_conf)
            with open(store_risultati, 'a', newline='\n') as file:
                for i in range(0, len(y_pred)):
                    kswin_lof_model[i_m].add_element(y_conf[i])
                    writer = csv.writer(file)
                    writer.writerow([y_pred[i],y_conf[i],i_m])
                    predizioni.append(y_pred[i])
            n_modelli_usati+=1
        else:
            print(f"Batch:{n_batch}")
            
            for i in range(0, len(kswin_cf)):
                if kswin_cf[i]==True:
                    f=1
            if f==1:
                n_conceptd_drift_dati+=1
                print(f"Rilevato concept drift")
                for i in range(0, len(Lof_models)):
                    print(f"Predizione modello:{i}")
                    y_pred_val,y_conf_val=Lof_models[i].predict(dati,return_confidence=True)
                    print(f"test modello:{i}")
                    for j in range(0, len(y_conf_val)):
                        kswin_lof_model[i].add_element(y_conf_val[j]) #dovra essere fatto append
                        if kswin_lof_model[i].detected_change()==True:
                            kswin_lof_model_cf[i]=True
                    if kswin_lof_model_cf[i]==True:
                        kswin_lof_model[i].reset()
                        print(f'Il modello {i} non è quello del concetto, viene resettato')
                        print(len(finestra_del_concetto[i]))
                        print(f"Sto ricaricando il concetto al modello:{i}")
                        for t in range(0, len(finestra_del_concetto[i])):
                            kswin_lof_model[i].add_element(finestra_del_concetto[i][t]) #reset con il concetto del modello
                        kswin_lof_model_cf[i]=False
                    else:
                        g=0
                        i_m=i #sappiamo quale modello usare, quello per cui non è stato rilevato cd
                        break
                    g=1
                    
                    
                if g==0: #sappiamo quale modello usare, quello per cui non è stato rilevato cd
                    print(f"Utilizzo un modello che già avevo:{i_m}")
                    n_riutilizzo_modello+=1
                    y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                    with open(store_risultati, 'a', newline='\n') as file:
                        for i in range(0, len(y_pred)):
                            writer = csv.writer(file)
                            writer.writerow([y_pred[i],y_conf[i],i_m])
                            predizioni.append(y_pred[i])
                    
                    
                else: #creo nuovo modello addestrandolo sull'ultima finestra di dati
                    print("Devo creare un nuovo modello")
                    if len(Lof_models)<5: #mantengo un massimo di 5 modelli
                        kswin_lof_model_cf.append(False)
                        kswin_lof_model.append(sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W))
                        Lof_models.append(LOF(n_neighbors=n_neig, algorithm='ball_tree', leaf_size=30, metric='minkowski', p=3, metric_params=None, contamination=0.1, n_jobs=1))
                        Lof_models[-1].fit(dati)
                        i_m=len(Lof_models)-1
                        y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                        finestra_del_concetto.append(y_conf)
                        print(f"I modelli adesso sono:{len(Lof_models)}")
                        
                    else: # eventualmente elimino il più vecchio
                        print("I modelli erano 5, ho sostituito il piu vecchio")
                        kswin_lof_model_cf[0]=False
                        kswin_lof_model[0]=sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W)
                        Lof_models[0]=LOF(n_neighbors=n_neig, algorithm='ball_tree', leaf_size=30, metric='minkowski', p=3, metric_params=None, contamination=0.1, n_jobs=1)
                        Lof_models[0].fit(dati)
                        i_m=0
                        y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                        finestra_del_concetto[i_m]=y_conf
                    
                    with open(store_risultati, 'a', newline='\n') as file:
                        for i in range(0, len(y_pred)):
                            kswin_lof_model[i_m].add_element(y_conf[i])
                            writer = csv.writer(file)
                            writer.writerow([y_pred[i],y_conf[i],i_m])
                            predizioni.append(y_pred[i])
                    n_modelli_usati+=1
            else: #continuo ad usare l'ultimo modello selezionato
                
            print(f"Concept drift non rilevato, modello in uso:{i_m}")
            y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
            with open(store_risultati, 'a', newline='\n') as file:
                for i in range(0, len(y_pred)):
                    writer = csv.writer(file)
                    writer.writerow([y_pred[i],y_conf[i],i_m])
                    predizioni.append(y_pred[i])
                
    else:
        print("Fine")
        fine=True

'''

In [ ]:
start_time=time.time()*1000
while fine!=True:
    n_batch+=1
    dati_cd=data_stream_cd.next_sample(batch_size=len_W)[0]
    dati=data_stream.next_sample(batch_size=len_W)[0]
    for i in range(0,len(kswin_cf)):
        kswin_cf[i]=False
    f=0

    if len(dati_cd)!=0:
        for i in range(0, len(kswin)):
            for dato in dati_cd:
                kswin[i].add_element(dato[i])
                if kswin[i].detected_change()==True:
                    kswin_cf[i]=True
        for i in range(0, len(kswin_cf)):
            if kswin_cf[i]==True:
                print(f'Drift zone has been detected on feature {i} in batch {n_batch} with kswin')
            else:
                print(f'No Drift zone has been detected on feature {i} in batch {n_batch} with kswin')

        if n_batch==1:
            print(f"Primo batch, creazione modello di base:{i_m}")
            i_m=0
            kswin_lof_model_cf.append(False)
            kswin_lof_model.append(sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W))
            Lof_models.append(LOF(n_neighbors=n_neig, algorithm='ball_tree', leaf_size=30, metric='minkowski', p=3, metric_params=None, contamination=0.1, n_jobs=1))
            Lof_models[i_m].fit(dati)
            y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
            finestra_del_concetto.append(y_conf)
            with open(store_risultati, 'a', newline='\n') as file:
                for i in range(0, len(y_pred)):
                    kswin_lof_model[i_m].add_element(y_conf[i])
                    writer = csv.writer(file)
                    writer.writerow([y_pred[i],y_conf[i],i_m])
                    predizioni.append(y_pred[i])
            n_modelli_usati+=1
        else:
            print(f"Batch:{n_batch}")
            for i in range(0, len(kswin_cf)):
                if kswin_cf[i]==True:
                    f=1
            if f==1:
                n_conceptd_drift_dati+=1
                print(f"Rilevato concept drift")
                for i in range(0, len(Lof_models)):
                    print(f"Predizione modello:{i}")
                    y_pred_val,y_conf_val=Lof_models[i].predict(dati,return_confidence=True)
                    print(f"test modello:{i}")
                    for j in range(0, len(y_conf_val)):
                        kswin_lof_model[i].add_element(y_conf_val[j]) #dovra essere fatto append
                        if kswin_lof_model[i].detected_change()==True:
                            kswin_lof_model_cf[i]=True
                    if kswin_lof_model_cf[i]==True:
                        kswin_lof_model[i].reset()
                        print(f'Il modello {i} non è quello del concetto, viene resettato')
                        print(len(finestra_del_concetto[i]))
                        print(f"Sto ricaricando il concetto al modello:{i}")
                        for t in range(0, len(finestra_del_concetto[i])):
                            kswin_lof_model[i].add_element(finestra_del_concetto[i][t]) #reset con il concetto del modello
                        kswin_lof_model_cf[i]=False
                    else:
                        g=0
                        i_m=i #sappiamo quale modello usare, quello per cui non è stato rilevato cd
                        break
                    g=1
                    
                    
                if g==0: #sappiamo quale modello usare, quello per cui non è stato rilevato cd
                    print(f"Utilizzo un modello che già avevo:{i_m}")
                    n_riutilizzo_modello+=1
                    y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                    with open(store_risultati, 'a', newline='\n') as file:
                        for i in range(0, len(y_pred)):
                            writer = csv.writer(file)
                            writer.writerow([y_pred[i],y_conf[i],i_m])
                            predizioni.append(y_pred[i])
                    
                    
                else: #creo nuovo modello addestrandolo sull'ultima finestra di dati
                    print("Devo creare un nuovo modello")
                    if len(Lof_models)<max_model: #mantengo un massimo di 5 modelli
                        kswin_lof_model_cf.append(False)
                        kswin_lof_model.append(sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W))
                        Lof_models.append(LOF(n_neighbors=n_neig, algorithm='ball_tree', leaf_size=30, metric='minkowski', p=3, metric_params=None, contamination=0.1, n_jobs=1))
                        Lof_models[-1].fit(dati)
                        i_m=len(Lof_models)-1
                        y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                        finestra_del_concetto.append(y_conf)
                        print(f"I modelli adesso sono:{len(Lof_models)}")
                        
                    else: # eventualmente elimino il più vecchio
                        print(f"I modelli erano {len(Lof_models)}, ho sostituito il piu vecchio")
                        kswin_lof_model_cf[0]=False
                        kswin_lof_model[0]=sk.drift_detection.KSWIN(alpha=0.005, window_size=len_CD-1, stat_size=len_W)
                        Lof_models[0]=LOF(n_neighbors=n_neig, algorithm='ball_tree', leaf_size=30, metric='minkowski', p=3, metric_params=None, contamination=0.1, n_jobs=1)
                        Lof_models[0].fit(dati)
                        i_m=0
                        y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                        finestra_del_concetto[i_m]=y_conf
                    
                    with open(store_risultati, 'a', newline='\n') as file:
                        for i in range(0, len(y_pred)):
                            kswin_lof_model[i_m].add_element(y_conf[i])
                            writer = csv.writer(file)
                            writer.writerow([y_pred[i],y_conf[i],i_m])
                            predizioni.append(y_pred[i])
                    n_modelli_usati+=1
            else: #continuo ad usare l'ultimo modello selezionato
                print(f"Concept drift non rilevato, modello in uso:{i_m}")
                y_pred,y_conf=Lof_models[i_m].predict(dati,return_confidence=True)
                with open(store_risultati, 'a', newline='\n') as file:
                    for i in range(0, len(y_pred)):
                        writer = csv.writer(file)
                        writer.writerow([y_pred[i],y_conf[i],i_m])
                        predizioni.append(y_pred[i])
                
    else:
        print("Fine")
        fine=True
stop_time=time.time()*1000


In [ ]:
#Si hanno i modelli usati, la loro successione e i dati relativi al loro concetto, si possono mostare i grafici
'''import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print(finestra_del_concetto[4])

fig=plt.figure()
for i in range(0, len(finestra_del_concetto)):
    df = pd.DataFrame(finestra_del_concetto[i], columns = ['Column_A','Column_B','Column_C'])

    ax= fig.add_subplot(i+1, projection='3d')
    x= finestra_del_concetto[i]

'''
print(n_batch)
print(n_conceptd_drift_dati)
print(n_riutilizzo_modello)
print(n_modelli_usati)
print(f"Tempo di esecuzione in millisecondi: {stop_time-start_time}")

In [ ]:
from sklearn.metrics import f1_score, classification_report, accuracy_score
import scikitplot as skplt 
import matplotlib.pyplot as plt

predizioni_np=np.array(predizioni)
verita_np=np.array(verita_list)



acc=accuracy_score(verita_np,predizioni_np)
print(f"ACC:{acc}")
f1_M=f1_score(verita_np,predizioni_np,average="macro")
print(f"MACRO:{f1_M}")
f1_W=f1_score(verita_np,predizioni_np,average="weighted")
print(f"WEIGH:{f1_W}")

report=classification_report(verita_np,predizioni_np)
print(report)


skplt.metrics.plot_confusion_matrix(verita_np,predizioni_np, normalize=True, cmap="gray_r", title=" " )
plt.show()
plt.savefig()
skplt.metrics.plot_confusion_matrix(verita_np,predizioni_np)
plt.show()
